In [54]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import time

# 用pd.read_html抓取資料表格

In [2]:
station_info = pd.read_csv('station_info.csv')

In [3]:
station_info.drop('Unnamed: 0',axis = 1,inplace=True)   #axis:去行（0）或列（1）

In [4]:
station_info

,466880,466900,466910,466920,466930,466940,466950,466990,467050,467060,...,C1V600,C1V780,C1X040,C1Z030,C1Z040,C1Z110,C1Z120,C1Z130,C1Z140,C1Z240
0,板橋,淡水,鞍部,臺北,竹子湖,基隆,彭佳嶼,花蓮,新屋,蘇澳,...,藤枝,多納林道,東原,紅葉,立山,三棧,壽豐,銅門,荖溪,中平林道
1,BANQIAO,TAMSUI,ANBU,TAIPEI,ZHUZIHU,KEELUNG,PENGJIAYU,HUALIEN,XINWU,SU-AO,...,Tengzhi,Duonalindao,Dongyuan,Hongye,Lishan,Sanzhan,Shoufeng,Tongmen,Laoxi,Zhongpinglindao
2,新北市,新北市,臺北市,臺北市,臺北市,基隆市,基隆市,花蓮縣,桃園市,宜蘭縣,...,高雄市,高雄市,臺南市,花蓮縣,花蓮縣,花蓮縣,花蓮縣,花蓮縣,花蓮縣,花蓮縣
3,%25E6%259D%25BF%25E6%25A9%258B,%25E6%25B7%25A1%25E6%25B0%25B4,%25E9%259E%258D%25E9%2583%25A8,%25E8%2587%25BA%25E5%258C%2597,%25E7%25AB%25B9%25E5%25AD%2590%25E6%25B9%2596,%25E5%259F%25BA%25E9%259A%2586,%25E5%25BD%25AD%25E4%25BD%25B3%25E5%25B6%25BC,%25E8%258A%25B1%25E8%2593%25AE,%25E6%2596%25B0%25E5%25B1%258B,%25E8%2598%2587%25E6%25BE%25B3,...,%25E8%2597%25A4%25E6%259E%259D,%25E5%25A4%259A%25E7%25B4%258D%25E6%259E%2597%...,%25E6%259D%25B1%25E5%258E%259F,%25E7%25B4%2585%25E8%2591%2589,%25E7%25AB%258B%25E5%25B1%25B1,%25E4%25B8%2589%25E6%25A3%25A7,%25E5%25A3%25BD%25E8%25B1%2590,%25E9%258A%2585%25E9%2596%2580,%25E8%258D%2596%25E6%25BA%25AA,%25E4%25B8%25AD%25E5%25B9%25B3%25E6%259E%2597%...
4,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...,https://e-service.cwb.gov.tw/HistoryDataQuery/...


In [5]:
stationIndexList = station_info.columns

# [降雨量, 溫度] = getClimate(site,date,mode)
## 需先執行
- station_info = pd.read_csv('station_info.csv')
- station_info.drop('Unnamed: 0',axis = 1,inplace=True)
- stationIndexList = station_info.columns

## site : 地區 (ex:臺北市)
## date : 日期 (ex:2018-06-05)
## mode=1 : 較慢但較準，採取人工測站與無人測站，且加入delay防鎖IP
## mode=2 : 較快但不準，只採取人工測站，未加入delay

In [82]:
def getClimate(site,date,mode):
    
    precpAvgList = list()
    tempAvgList = list()
    
    #台-->臺
    if site[0] == '台':
        site = '臺'+site[1:] #字串無法直接指定, ex:site[k] = '臺'
            
    if mode == 1: 
        for i in range(len(stationIndexList)):       
            
            #此地區每個測站
            if station_info[stationIndexList[i]][2] == site:
                
                #需要delay才不會被擋
                time.sleep(0.11) 
                
                #去指定網址爬取
#                 print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
                
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                    
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
                    
        #算出所有測站平均回傳 
#         print(precpAvgList)
#         print(tempAvgList)
        
        return np.average(precpAvgList), np.average(tempAvgList)
    
    else:
        for i in range(len(stationIndexList)):
            
            #此地區每個人工測站
            if (station_info[stationIndexList[i]][2] == site) and stationIndexList[i].isdigit():
                
                #去指定網址爬取
#                 print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
    
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
#             else:
#                 break
        #算出人工測站平均回傳    
#         print(precpAvgList)
#         print(tempAvgList)
        return np.average(precpAvgList), np.average(tempAvgList)

## example

In [86]:
getClimate('桃園市','2018-03-06',1)

(0.0, 16.376923076923074)

In [87]:
getClimate('桃園市','2018-03-06',2)

(0.0, 17.2)

- 應該沒差多少吧